In [1]:
""" 
1- Read data into a dataframe (mma)
2- Separate data into train/test (80/20) (mma_train , mma_test)
3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)
4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)
5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data
6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq
7- Add DepartmentID|AisleID to solution table by joining with main table
8- Create SubstituteIDs for each ProductID
    >> For each AisleID|DepartmentID group:
        1)Select all ProductIDs | ProductNames under Aisle|Department
        2)Do a similarity analysis, and try to cluster similar product names
        3)On each cluster, compare frequency of product in orders, and select the most frequent product as SubstituteID
            Left Join with frequency table, and compare
        4)For outliers (products that are not similar to any other product), copy ProductID as SubstituteID
        5)Append ProductIDs|SubstituteIDs to Substitute table
9- Add SubstituteIDs to solution table by joining with Substitute table
10- Prepare objective function as a method
11- Prepare constraints as a method
12- Configure optimization with Objective, Constraints and Solution Table
13- Run optimization
14- Export solution table as csv
15- Test solution table with test data
16- Calculate score against metrics
"""

' \n1- Read data into a dataframe (mma)\n2- Separate data into train/test (80/20) (mma_train , mma_test)\n3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)\n4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)\n5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data\n6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq\n7- Add DepartmentID|AisleID to solution table by joining with main table\n8- Create SubstituteIDs for each ProductID\n    >> For each AisleID|DepartmentID group:\n        1)Select all ProductIDs | ProductNames under Aisle|Department\n        2)Do a similarity analysis, and try to cluster similar product names\n        3)On each cluster, compare frequency of product in orders, and select the most frequent product as Substit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sklearn as sk
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import math
import random
import itertools
import time
import pulp
import os
import warnings
import csv
warnings.filterwarnings('ignore')


### 1- Read data into a dataframe (mma)

In [3]:
# Reasd the data
mma = pd.read_csv('data/mma_mart.csv')
mma.head()

,order_id,product_id,product_name,aisle_id,aisle,department_id,department
0,1,49302,Bulgarian Yogurt,120,yogurt,16,dairy eggs
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs
2,1,10246,Organic Celery Hearts,83,fresh vegetables,4,produce
3,1,49683,Cucumber Kirby,83,fresh vegetables,4,produce
4,1,43633,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods


In [4]:
# Check lenght of order_ids
mma.nunique()

order_id         97833
product_id       35070
product_name     35070
aisle_id           134
aisle              134
department_id       21
department          21
dtype: int64

### 2- Separate data into Train/Test

In [5]:
mma_train = mma[mma['order_id'] <= 80000]
mma_test  = mma[mma['order_id'] > 80000]

### 3- Calculate frequency in train data | test data of each product over all orders (mma-train-freq | mma-test-freq)

In [6]:
# Count frequency of products in train and test data
f_counts_train = mma_train['product_id'].value_counts()
f_counts_test = mma_test['product_id'].value_counts()

In [7]:
# Create dataframes for frequency counts

## Train   
f_counts_train_df = f_counts_train.to_frame().reset_index()
f_counts_train_df.columns = ['product_id', 'frequency']
## Test
f_counts_test_df = f_counts_test.to_frame().reset_index()
f_counts_test_df.columns = ['product_id', 'frequency']

In [8]:
mma_train_freq = f_counts_train_df.sort_values(by=['frequency'], ascending=False)
mma_test_freq = f_counts_test_df.sort_values(by=['frequency'], ascending=False)

### 4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?)

In [9]:
solution_df = pd.DataFrame(columns=['ProductID', 'SubstituteID', 'PFrequency', 'DepartmentID', 'AisleID', 'Selected?'])

### 5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data

In [10]:
solution_df['ProductID'] = mma['product_id'].unique()
solution_df.sort_values(by=['ProductID'], inplace=True)
solution_df.head()

,ProductID,SubstituteID,PFrequency,DepartmentID,AisleID,Selected?
4788,1,NaN,NaN,NaN,NaN,NaN
17643,2,NaN,NaN,NaN,NaN,NaN
8823,3,NaN,NaN,NaN,NaN,NaN
18351,4,NaN,NaN,NaN,NaN,NaN
31588,8,NaN,NaN,NaN,NaN,NaN


### 6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq

In [11]:
solution_df = pd.merge(solution_df, mma_train_freq, how='left', left_on=['ProductID'], right_on=('product_id'))
solution_df.drop(columns=['product_id'], inplace=True)
solution_df.drop(columns=['PFrequency'], inplace=True)
solution_df.rename(columns={'frequency':'PFrequency'}, inplace=True)
solution_df.head()

,ProductID,SubstituteID,DepartmentID,AisleID,Selected?,PFrequency
0,1,NaN,NaN,NaN,NaN,51.0
1,2,NaN,NaN,NaN,NaN,2.0
2,3,NaN,NaN,NaN,NaN,4.0
3,4,NaN,NaN,NaN,NaN,6.0
4,8,NaN,NaN,NaN,NaN,1.0


### 7- Add DepartmentID|AisleID to solution table by joining with main table

In [12]:
mma_train_unique = mma_train.drop_duplicates(subset=['product_id', 'department_id', 'aisle_id'])
mma_train_unique.nunique()

order_id         17575
product_id       33171
product_name     33171
aisle_id           134
aisle              134
department_id       21
department          21
dtype: int64

In [13]:
# Checking if product_id|department_id|aisle_id is unique
mma_train.groupby(['product_id', 'department_id', 'aisle_id']).nunique()

,,,order_id,product_name,aisle,department
product_id,department_id,aisle_id,,,,
1,19,61,51,1,1,1
2,13,104,2,1,1,1
3,7,94,4,1,1,1
4,1,38,6,1,1,1
8,1,116,1,1,1,1
...,...,...,...,...,...,...
49681,1,38,1,1,1,1
49683,4,83,2354,1,1,1
49685,1,42,2,1,1,1


In [14]:
solution_df = pd.merge(solution_df, mma_train_unique, how='left', left_on=['ProductID'], right_on=('product_id'))
solution_df.drop(columns=['product_id'], inplace=True)
solution_df.head()

,ProductID,SubstituteID,DepartmentID,AisleID,Selected?,PFrequency,order_id,product_name,aisle_id,aisle,department_id,department
0,1,NaN,NaN,NaN,NaN,51.0,1107.0,Chocolate Sandwich Cookies,61.0,cookies cakes,19.0,snacks
1,2,NaN,NaN,NaN,NaN,2.0,12209.0,All-Seasons Salt,104.0,spices seasonings,13.0,pantry
2,3,NaN,NaN,NaN,NaN,4.0,2998.0,Robust Golden Unsweetened Oolong Tea,94.0,tea,7.0,beverages
3,4,NaN,NaN,NaN,NaN,6.0,13379.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,38.0,frozen meals,1.0,frozen
4,8,NaN,NaN,NaN,NaN,1.0,66768.0,Cut Russet Potatoes Steam N' Mash,116.0,frozen produce,1.0,frozen


In [15]:
# Given aisle_id and department_id are 1-1 | N-1, we can drop department_id and keep aisle_id as the lowest granularity
solution_df.drop(columns=['order_id', 'aisle','DepartmentID', 'AisleID', 'department_id'], inplace=True)
solution_df.head()

,ProductID,SubstituteID,Selected?,PFrequency,product_name,aisle_id,department
0,1,NaN,NaN,51.0,Chocolate Sandwich Cookies,61.0,snacks
1,2,NaN,NaN,2.0,All-Seasons Salt,104.0,pantry
2,3,NaN,NaN,4.0,Robust Golden Unsweetened Oolong Tea,94.0,beverages
3,4,NaN,NaN,6.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,38.0,frozen
4,8,NaN,NaN,1.0,Cut Russet Potatoes Steam N' Mash,116.0,frozen


### 8- Create SubstituteIDs for each ProductID

In [16]:
""" This step was built based on the tutorial from the link below, to cluster product names and find similar products """
# Source : https://medium.com/towards-data-science/clustering-product-names-with-python-part-1-f9418f8705c8
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import plotly.express as px

#Libraries for preprocessing
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import webcolors

#Download once if using NLTK for preprocessing
import nltk

#Libraries for vectorisation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from fuzzywuzzy import fuzz

#Libraries for clustering
from sklearn.cluster import KMeans

In [17]:
#### METHODS ####

#Stem and make lower case
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return ' '.join(stem_sentence)

#Plot topics function. Code from: https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(6, 5, figsize=(30, 30), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)
    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [18]:
# # Select all products under aisle
# aisle_products = solution_df[solution_df['aisle_id'] == 1]
# # Select all product names under aisle
# aisle_products_names_1 = aisle_products['product_name']
# # Remove stopwords, punctuation, and numbers
# aisle_products_names_2 = [remove_stopwords(x)\
#     .translate(str.maketrans('','',string.punctuation))\
#     .translate(str.maketrans('','',string.digits))\
#     for x in aisle_products_names_1]
# # Stemming and making words lower case
# aisle_products_names_3 = pd.Series([stemSentence(x) for x in aisle_products_names_2])
# # Removing Color Names
# colors = list(webcolors.CSS3_NAMES_TO_HEX)
# colors = [stemSentence(x) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
#                                      'olive','orange','plum','salmon','tomato','wheat')]
# aisle_product_names_4 = [' '.join([x for x in string.split() if x not in colors]) for string in aisle_products_names_3]
# # Vectorizing
# ## Bag of Words
# vectorizer_cv = CountVectorizer(analyzer='word')
# X_cv = vectorizer_cv.fit_transform(aisle_product_names_4)
# ## TF-IDF
# vectorizer_wtf = TfidfVectorizer(analyzer='word')
# X_wtf = vectorizer_wtf.fit_transform(aisle_product_names_4)
# ## LDA
# lda = LatentDirichletAllocation(n_components=30, learning_decay=0.9)
# X_lda = lda.fit(X_cv)
# ## FuzzyWuzzy
# X_fuzz = pd.crosstab([aisle_product_names_4,aisle_product_names_4],aisle_product_names_4).apply(lambda col: [fuzz.token_sort_ratio(col.name, x) 
#                                                                    for x in col.index.get_level_values(1)])

In [19]:
##Show topics
# n_top_words = 5
# feature_names = vectorizer_cv.get_feature_names_out()
# plot_top_words(X_lda, feature_names, n_top_words, '')


In [20]:
# #Test increments of 100 clusters using elbow method
# sse={}
# for k in np.arange(1,110,1):
#     kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X_cv)
#     sse[k] = kmeans.inertia_
# plt.plot(list(sse.keys()),list(sse.values()))
# plt.xlabel('Values for K')
# plt.ylabel('SSE')
# plt.show()

In [21]:
# pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())

In [22]:
# a = aisle_products_names_1.to_frame()
# b= pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())
# b.index = a.index
# c = pd.concat([a,b], axis=1)

In [23]:
# #Create 20 clusters
# kmeans = KMeans(n_clusters=np.round(np.shape(aisle_products_names_1)[0]/5).astype(int))
# kmeans.fit(X_cv)
# aisle_products_names_1.reset_index(drop=True, inplace=True)
# result = pd.concat([aisle_products_names_1, pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())], axis=1)
# result['cluster'] = kmeans.predict(X_cv)

In [24]:
# result.iloc[:,[0,-1]]

In [25]:
# d = pd.merge(solution_df, result.iloc[:,[0,-1]], how='left', left_on=['product_name'], right_on=('product_name'))
# d[d['product_name'] == 'Italian Pasta Salad']

In [26]:
# np.round(np.shape(aisle_products_names_1)[0]/5).astype(int)

In [27]:
""" 
For each AisleID|DepartmentID group:
    1)Select all ProductIDs | ProductNames under Aisle|Department
    2)Do a similarity analysis, and try to cluster similar product names
    3)On each cluster, compare frequency of product in orders, and select the most frequent product as SubstituteID
        Left Join with frequency table, and compare
    4)For outliers (products that are not similar to any other product), copy ProductID as SubstituteID
    5)Append ProductIDs|SubstituteIDs to Substitute table
"""

for aisle in solution_df.drop_duplicates(['aisle_id'])['aisle_id']:
    # Select all products under aisle
    aisle_products = solution_df[solution_df['aisle_id'] == aisle]
    # Select all product names under aisle
    aisle_products_names_1 = aisle_products['product_name']
    
    # Check if there are products under aisle, if not, continue
    if len(aisle_products_names_1) in [0,1]:
        continue

    # Preprocessing
    ## Remove stopwords, punctuation, and numbers
    aisle_products_names_2 = [remove_stopwords(x)\
        .translate(str.maketrans('','',string.punctuation))\
        .translate(str.maketrans('','',string.digits))\
        for x in aisle_products_names_1]
    
    ## Stemming and making words lower case
    aisle_products_names_3 = pd.Series([stemSentence(x) for x in aisle_products_names_2])

    ## Removing Color Names
    colors = list(webcolors.CSS3_NAMES_TO_HEX)
    colors = [stemSentence(x) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
                                        'olive','orange','plum','salmon','tomato','wheat')]
    aisle_product_names_4 = [' '.join([x for x in string.split() if x not in colors]) for string in aisle_products_names_3]

    # Vectorizing
    ## Bag of Words
    vectorizer_cv = CountVectorizer(analyzer='word')
    X_cv = vectorizer_cv.fit_transform(aisle_product_names_4)
    ## TF-IDF
    vectorizer_wtf = TfidfVectorizer(analyzer='word')
    X_wtf = vectorizer_wtf.fit_transform(aisle_product_names_4)
    ## LDA
    lda = LatentDirichletAllocation(n_components=30, learning_decay=0.9)
    X_lda = lda.fit(X_cv)
    ## FuzzyWuzzy
    X_fuzz = pd.crosstab([aisle_product_names_4,aisle_product_names_4],aisle_product_names_4).apply(lambda col: [fuzz.token_sort_ratio(col.name, x) 
                                                                    for x in col.index.get_level_values(1)])
    
    # Clustering
    ## Create (NumberOfProductsAtAisle/5) clusters, and assign each product to a cluster
    if np.shape(aisle_products_names_1)[0]/5 <= 1:
        n = 2
    else:
        n = np.round(np.shape(aisle_products_names_1)[0]/5).astype(int)
    kmeans = KMeans(n_clusters=n)
    kmeans.fit(X_cv)
    
    ## Create a dataframe with product names and cluster number , reindex to match with solution_df
    a = aisle_products_names_1.to_frame()
    b= pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())
    b.index = a.index
    result = pd.concat([a,b], axis=1)
    result[f'cluster_{aisle}'] = kmeans.predict(X_cv)

    # Joining with solution_df
    solution_df = pd.merge(solution_df, result[['product_name',f'cluster_{aisle}']], how='left')

In [28]:
solution_df.head()

,ProductID,SubstituteID,Selected?,PFrequency,product_name,aisle_id,department,cluster_61.0,cluster_104.0,cluster_94.0,...,cluster_33.0,cluster_102.0,cluster_86.0,cluster_39.0,cluster_113.0,cluster_35.0,cluster_76.0,cluster_118.0,cluster_71.0,cluster_32.0
0,1,NaN,NaN,51.0,Chocolate Sandwich Cookies,61.0,snacks,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,2.0,All-Seasons Salt,104.0,pantry,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,4.0,Robust Golden Unsweetened Oolong Tea,94.0,beverages,NaN,NaN,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,6.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,38.0,frozen,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,NaN,NaN,1.0,Cut Russet Potatoes Steam N' Mash,116.0,frozen,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 9- Add SubstituteIDs to solution table by joining with Substitute table

In [68]:
for aisle in solution_df.drop_duplicates(['aisle_id'])['aisle_id']:
    if math.isnan(aisle):
        continue
    
    # Select all products under aisle
    aisle_products = solution_df[solution_df['aisle_id'] == aisle]
    # Select all product names under aisle
    aisle_products_cluster = aisle_products[['product_name', 'aisle_id', f'cluster_{aisle}', 'PFrequency']]
    # Select all clusters under aisle
    aisle_clusters = aisle_products_cluster.drop_duplicates([f'cluster_{aisle}'])

    for cluster in aisle_clusters[f'cluster_{aisle}']:
        # Select all products under cluster
        cluster_products = aisle_products_cluster[aisle_products_cluster[f'cluster_{aisle}'] == cluster]
        # Select all product names under cluster
        cluster_products_names = cluster_products['product_name']
        # Select all product frequencies under cluster
        cluster_products_freq = cluster_products['PFrequency']  
        # Check if there are products under cluster, if not, continue
        if len(cluster_products_names) in [0,1]:
            continue    

        # Select the product with highest frequency under cluster
        cluster_products_freq_max = cluster_products_freq.max()
        cluster_products_freq_max_index = cluster_products_freq.idxmax()
        cluster_products_freq_max_name = cluster_products_names[cluster_products_freq_max_index]
        cluster_products_freq_max_id = solution_df[solution_df['product_name'] == cluster_products_freq_max_name]['ProductID'].values[0]    
        
        # Append ProductID|SubstituteID to Substitute table
        solution_df.loc[solution_df[f'cluster_{aisle}'] == cluster, 'SubstituteID'] = cluster_products_freq_max_id

In [69]:
# # Export to a file
# solution_df.to_csv('data/solution_df_substitute2.csv', index=False)

### 10- Prepare objective function as a method

,ProductID,SubstituteID,Selected?,PFrequency,product_name,aisle_id,department,cluster_61.0,cluster_104.0,cluster_94.0,...,cluster_33.0,cluster_102.0,cluster_86.0,cluster_39.0,cluster_113.0,cluster_35.0,cluster_76.0,cluster_118.0,cluster_71.0,cluster_32.0
0,1,12481,NaN,51.0,Chocolate Sandwich Cookies,61.0,snacks,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,48437,NaN,2.0,All-Seasons Salt,104.0,pantry,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,NaN,4.0,Robust Golden Unsweetened Oolong Tea,94.0,beverages,NaN,NaN,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,10637,NaN,6.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,38.0,frozen,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,34466,NaN,1.0,Cut Russet Potatoes Steam N' Mash,116.0,frozen,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 11- Prepare constraints as a method

In [ ]:
def freezer_constraint(df):
    """
    The freezer constraint limits the amount of freezer products on the resulting list to a maximum of 100
    """

    if df.loc[df['department'] == 'frozen']['Selected?'].sum() <= 100:
        return True
    else:
        return False
    
def refrigerator_constraint(df):
    if df.loc[(df['department'] == 'meat seafood' ) | (df['department'] == 'dairy eggs')]['Selected?'].sum() <= 100:
        return True
    else:
        return False

def selections_constraint(df):
    if df['Selected'].sum() <= 1000:
        return True
    else:
        return False

### 12- Configure optimization with Objective, Constraints and Solution Table

### 13- Run optimization